In [22]:
import html
import re

def fetch_file(path):
    f = open(path, 'r', encoding="utf-8")
    content = html.unescape(f.read())
    # print(content)
    search = re.search("<span><p class=............>(.*)</p></span>", content)
    # search = re.search("<span><p class=(.*)</p></span>", content)
    if search != None:
        body = re.sub("<.*?>", "", search.group(1))
        return body
    else:
        return ""

In [3]:
# development tests for clean_file
# should be able to extract body text, and any other features that might be nice.

path = "..\Adzhubei__Aleksei\March_12__1962\html\Alexei_Adzhubei-s_Account_of_His_Visit_to_Washington_to_the_Central_Committee_of_the_Communist_Party_of_the_Soviet_Union__0.html"
clean_file(path)


FileNotFoundError: [Errno 2] No such file or directory: '..\\Adzhubei__Aleksei\\March_12__1962\\html\\Alexei_Adzhubei-s_Account_of_His_Visit_to_Washington_to_the_Central_Committee_of_the_Communist_Party_of_the_Soviet_Union__0.html'

In [23]:
# Obtaining metadata from folder structure.

from pathlib import Path
import os

problem_files = []
output = []
n = 0

for subdir, dirs, files in os.walk("."):
    for file in files:
        filepath = os.path.join(subdir, file)

        if filepath.endswith(".html"):
            n += 1
            patharr = filepath.split(os.sep)
            #print(filepath)
            #print('author_name: ' + patharr[1])
            #print('date: ' + patharr[2])
            #print('title: ' + patharr[4][:-5]) 
            
            current_file_info = [patharr[1], patharr[2], patharr[4][:-5], ""]

            try:
                current_file_info[3] = fetch_file(filepath)
            except (FileNotFoundError):
                problem_files.append(("file not found", filepath))
            except (UnicodeDecodeError):
                problem_files.append(("unicode decode issues", filepath))
            
            output.append(current_file_info)

In [24]:
# Report on problem files
print("Total files processed: " + str(n))
print("Files with errors: " + str(len(problem_files)))

Total files processed: 778
Files with errors: 3


In [25]:
# Import cleaned data into dataframe

import pandas as pd

df = pd.DataFrame.from_records(output)
df

,0,1,2,3
0,Szarvas__Pal,August_17__1955,Report__Embassy_of_Hungary_in_North_Korea_to_t...,"On 29 July 1955, at 4:30 p.m., I returned the ..."
1,Communist_Party_of_Czechoslovakia_(CPCz),July__1972,Information_from_Consultative_Meeting_about_Ch...,
2,Jelen__Bolesław,March_24__1962,Telegram_from_Polish_Embassy_in_Havana_(Jeleń)...,Ciphergram No. 4098Dispatched from Havana on 0...
3,Mao__Zedong__1893-1976,June_21__1951,Telegram_from_Mao_Zedong_to_I_V__Stalin_0,SECOND MAIN ADMINISTRATION OF THE GENERAL STAF...
4,Mao__Zedong__1893-1976,July_11__1951,Cable_No__21680__Mao_Dong_to_Cde__Filippov_[St...,SECOND MAIN DIRECTORATE OF THE GENERAL STAFF O...
...,...,...,...,...
773,Sabri__Ali_,June_19__1965,Record_of_Conversation_between_Premier_Zhou_an...,Secret Document 618 Department of West Asian ...
774,Værnø__Oscar,November_02__1976,Telegram_from_Norwegian_Embassy_in_Washington_...,ROYAL NORWEGIAN EMBASSYWASHINGTON D.C.2. NOVEM...
775,China__Yanbian_Military_District_Party_Committee,September_08__1966,Cable_from_the_Yanbian_Military_District_Party...,Jilin Province Military District Headquarters ...
776,Malik__G__J_,October_12__1947,Incoming_Cable_No__16__Malik_to_Cde__Stalin_0,STRICTLY SECRET CABLEFrom Moscow sent at 0350 ...


In [26]:
#preprocessing
df["secondary_data"] = df[3].map(lambda x: re.sub('[,\.!?]', '', x)).map(lambda x: x.lower())
df.head()

,0,1,2,3,secondary_data
0,Szarvas__Pal,August_17__1955,Report__Embassy_of_Hungary_in_North_Korea_to_t...,"On 29 July 1955, at 4:30 p.m., I returned the ...",on 29 july 1955 at 4:30 pm i returned the visi...
1,Communist_Party_of_Czechoslovakia_(CPCz),July__1972,Information_from_Consultative_Meeting_about_Ch...,,
2,Jelen__Bolesław,March_24__1962,Telegram_from_Polish_Embassy_in_Havana_(Jeleń)...,Ciphergram No. 4098Dispatched from Havana on 0...,ciphergram no 4098dispatched from havana on 03...
3,Mao__Zedong__1893-1976,June_21__1951,Telegram_from_Mao_Zedong_to_I_V__Stalin_0,SECOND MAIN ADMINISTRATION OF THE GENERAL STAF...,second main administration of the general staf...
4,Mao__Zedong__1893-1976,July_11__1951,Cable_No__21680__Mao_Dong_to_Cde__Filippov_[St...,SECOND MAIN DIRECTORATE OF THE GENERAL STAFF O...,second main directorate of the general staff o...


In [37]:
from nltk.corpus import stopwords
from bs4 import BeautifulSoup
import nltk

# load a list of stop words
nltk.download('stopwords')


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string 
        return: modified initial string
    """
    text = BeautifulSoup(text, 'html.parser').text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
df["secondary_data"] =df["secondary_data"].apply(clean_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ksatrayamaii/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
import gensim
from gensim.utils import simple_preprocess
def wordifier(line):
    for s in line:
        yield(gensim.utils.simple_preprocess(str(s), deacc=True))
word_list = list(wordifier(df.secondary_data.values.tolist()))

In [41]:
import gensim.corpora as corpora

# Create Dictionary
id2word = corpora.Dictionary(word_list)

# Create Corpus
texts = word_list

# Term Document Frequency
corpus = [id2word.doc2bow(t) for t in texts]


In [42]:
from pprint import pprint

# number of topics
num_topics = 10

# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)

# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]



[(0,
  '0.008*"korean" + 0.007*"party" + 0.006*"us" + 0.005*"also" + 0.005*"said" + '
  '0.004*"one" + 0.004*"time" + 0.004*"comrade" + 0.004*"il" + '
  '0.004*"chinese"'),
 (1,
  '0.010*"korea" + 0.010*"korean" + 0.009*"south" + 0.006*"north" + '
  '0.006*"party" + 0.005*"would" + 0.005*"said" + 0.005*"us" + 0.004*"time" + '
  '0.004*"one"'),
 (2,
  '0.007*"chinese" + 0.007*"people" + 0.006*"party" + 0.006*"korean" + '
  '0.005*"china" + 0.005*"korea" + 0.005*"soviet" + 0.005*"time" + 0.005*"us" '
  '+ 0.004*"countries"'),
 (3,
  '0.007*"korean" + 0.006*"us" + 0.006*"also" + 0.006*"comrade" + 0.005*"kim" '
  '+ 0.005*"countries" + 0.005*"korea" + 0.005*"party" + 0.004*"soviet" + '
  '0.004*"would"'),
 (4,
  '0.006*"korean" + 0.006*"party" + 0.006*"soviet" + 0.005*"il" + '
  '0.005*"comrade" + 0.005*"would" + 0.004*"time" + 0.004*"korea" + '
  '0.004*"kim" + 0.004*"china"'),
 (5,
  '0.007*"soviet" + 0.006*"would" + 0.006*"also" + 0.005*"us" + 0.005*"korean" '
  '+ 0.005*"korea" + 0.004

In [50]:
! pip install pyLDAvis

In [55]:
import pyLDAvis.gensim_models
import pickle 
import pyLDAvis

# Visualize the topics
pyLDAvis.enable_notebook()

LDAvis_data_filepath = os.path.join('../analysis/topic_'+str(num_topics))
if true :
    LDAvis_prepared = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)

# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)

pyLDAvis.save_html(LDAvis_prepared, '../analysis/topic_'+ str(num_topics) +'.html')

LDAvis_prepared


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.007909  0.007551       1        1  16.601087
8      0.007853  0.002563       2        1  14.249239
9      0.013933 -0.007200       3        1  13.840089
7     -0.001324  0.003631       4        1  13.706876
2     -0.006719  0.003469       5        1  13.275066
6      0.005332 -0.000444       6        1  11.490225
0     -0.005328 -0.004582       7        1   6.806106
5     -0.009445  0.001358       8        1   4.804698
3     -0.006015  0.000233       9        1   3.394472
4     -0.006197 -0.006579      10        1   1.832142, topic_info=         Term         Freq        Total Category  logprob  loglift
464    soviet  1383.000000  1383.000000  Default  30.0000  30.0000
345     party  1355.000000  1355.000000  Default  29.0000  29.0000
80    comrade  1062.000000  1062.000000  Default  28.0000  28.0000
272    korean  2467.000000  2467.000000  Default  27.0000  27.0000
537        us  1292.000000  1292.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
1007  chinese    16.529058  1257.163783  Topic10  -5.6990  -0.3318
428      said    15.397982  1076.150610  Topic10  -5.7699  -0.2472
1047  foreign    12.480816   790.755007  Topic10  -5.9799  -0.1491
321     north    13.665124  1246.648488  Topic10  -5.8893  -0.5137
967       war    12.212176   736.311058  Topic10  -6.0017  -0.0995

[814 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
10162      7  0.626459    accede
11013      1  0.498253  activist
11013      4  0.249127  activist
2245       1  0.079545   adviser
2245       2  0.053030   adviser
...      ...       ...       ...
2861       6  0.101810      zhou
2861       7  0.088414      zhou
2861       8  0.056263      zhou
2861       9  0.040188      zhou
2861      10  0.021434      zhou

[1928 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 9, 10, 8, 3, 7, 1, 6, 4, 5])